In [3]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [6]:
df = pd.read_csv('resource/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [8]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [11]:
print(df['headline'].isnull().values.any()) # null check

False


In [14]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.headline.values))
headline[:5] # Unknown은 null값은 아니지만 노이즈 데이터이므로 제거 요망

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [15]:
print('총 샘플의 개수: {}'.format(len(headline)))

총 샘플의 개수: 1324


In [17]:
headline = [word for word in headline if word != 'Unknown']
print('노이즈 제거 후 샘플의 개수: {}'.format(len(headline)))
headline[:5]

노이즈 제거 후 샘플의 개수: 1214


['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [21]:
# 구두점 제거와 단어의 소문자화
def repreprocessing(raw_sentence):
    preprocessed_sentence = raw_sentence.encode('utf8').decode('ascii', 'ignore')
    return ''.join(word for word in preprocessed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [29]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline) # 빈도수 기준 단어집합 생성
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 3494


In [27]:
sequences = list()

for sequence in preprocessed_headline:
    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sequence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [33]:
# 인덱스 대 단어 집합
index_to_word = {}
for key, value in tokenizer.word_index.items(): #인덱스를 단어로 바꾸기 위해 생성
    index_to_word[value] = key

print('빈도수 상위 582번째 단어: {}'.format(index_to_word[582]))

빈도수 상위 582번째 단어: offer


In [31]:
# 패딩 작업
max_len = max(len(l) for l in sequences)
print('샘플의 최대 크기: {}'.format(max_len))

샘플의 최대 크기: 24


In [32]:
# 최대 길이로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [39]:
# 맨 우측 단어만 레이블 분리
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

print(X[:3])
print(y[:3])

#원-핫 인코딩
y = to_categorical(y, num_classes=vocab_size)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[ 269  371 1115]


In [42]:
# 모델 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim)) # input: vocab_size,-> 단어 집합 크기 output: embedding_dim -> 임베딩 벡터 차원은 10
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax')) # 출력층
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 4s - loss: 7.6440 - accuracy: 0.0300 - 4s/epoch - 15ms/step
Epoch 2/200
244/244 - 3s - loss: 7.1171 - accuracy: 0.0305 - 3s/epoch - 11ms/step
Epoch 3/200
244/244 - 3s - loss: 6.9781 - accuracy: 0.0341 - 3s/epoch - 11ms/step
Epoch 4/200
244/244 - 3s - loss: 6.8527 - accuracy: 0.0402 - 3s/epoch - 11ms/step
Epoch 5/200
244/244 - 3s - loss: 6.7035 - accuracy: 0.0446 - 3s/epoch - 11ms/step
Epoch 6/200
244/244 - 3s - loss: 6.5335 - accuracy: 0.0477 - 3s/epoch - 11ms/step
Epoch 7/200
244/244 - 3s - loss: 6.3418 - accuracy: 0.0524 - 3s/epoch - 11ms/step
Epoch 8/200
244/244 - 3s - loss: 6.1373 - accuracy: 0.0587 - 3s/epoch - 11ms/step
Epoch 9/200
244/244 - 3s - loss: 5.9807 - accuracy: 0.0611 - 3s/epoch - 11ms/step
Epoch 10/200
244/244 - 3s - loss: 5.7631 - accuracy: 0.0672 - 3s/epoch - 11ms/step
Epoch 11/200
244/244 - 3s - loss: 5.5810 - accuracy: 0.0706 - 3s/epoch - 11ms/step
Epoch 12/200
244/244 - 3s - loss: 5.4124 - accuracy: 0.0763 - 3s/epoch - 11ms/step
Epoch 13/200


In [43]:
def sentence_generation(model, tokenizer, current_word, n): #모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            # 동일한 단어가 있으면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' ' + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [46]:
print(sentence_generation(model, tokenizer, 'i', 10))
print(sentence_generation(model, tokenizer, 'how', 10))
print(sentence_generation(model, tokenizer, 'wow', 10))

i disapprove of school vouchers can i still apply for them
how to make a crossword puzzle law is ignored in the
wow holds holds steady after tax cut is feather in trump
